In [23]:
import pandas as pd

In [24]:
df_dpm = pd.read_csv('dpm-experiments.csv')
df_dpm_heur = pd.read_csv('dpm-experiments-heur.csv')

df_dfm = pd.read_csv('dfm-experiments.csv')
df_dfm_heur = pd.read_csv('dfm-experiments-heur.csv')

In [25]:
def get_groups(x: str):
    y = x.split('-')[1]
    return ((int(y) - 1) // 10) + 1


def insert_group(df: pd.DataFrame) -> None:
    df.insert(0, 'group', df['name'].apply(get_groups))


insert_group(df_dpm)
insert_group(df_dpm_heur)
insert_group(df_dfm)
insert_group(df_dfm_heur)

In [26]:
df_dpm_heur.columns

Index(['group', 'name', 'neh', 'neh_ls', 'multistart', 'iga'], dtype='object')

In [27]:
def merge_metrics(df_bnb: pd.DataFrame, df_heur: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(
        df_bnb[['name', 'incumbent', 'lower_bound']],
        df_heur[['group', 'name', 'neh', 'neh_ls', 'multistart', 'iga']],
        on='name',
    )

    for col in ['neh', 'neh_ls', 'multistart', 'iga']:
        merged[f'rel_gap_{col}'] = (
            (merged[col] - merged['lower_bound']) / merged['lower_bound']
        ) * 100
        merged[f'rel_gap_{col}'] = merged[f'rel_gap_{col}'].apply(
            lambda x: round(x, 2)
        )
        merged[f'arpd_{col}'] = (
            (merged[col] - merged['incumbent']) / merged['incumbent']
        ) * 100
        merged[f'arpd_{col}'] = merged[f'arpd_{col}'].apply(
            lambda x: round(x, 2)
        )

    return merged


dpm_merged = merge_metrics(df_dpm, df_dpm_heur)
dfm_merged = merge_metrics(df_dfm, df_dfm_heur)

In [28]:
def agg_dataframe(df_out: pd.DataFrame) -> pd.DataFrame:
    return df_out.groupby('group', as_index=False).agg({
        'rel_gap_neh': 'mean',
        'arpd_neh': 'mean',
        'rel_gap_neh_ls': 'mean',
        'arpd_neh_ls': 'mean',
        'rel_gap_multistart': 'mean',
        'arpd_multistart': 'mean',
        'rel_gap_iga': 'mean',
        'arpd_iga': 'mean',
    })

In [29]:
def export_arpd(df: pd.DataFrame, filename: str) -> None:
    agg_df = agg_dataframe(df)
    for col in agg_df.columns:
        if col != 'group':
            agg_df[col] = agg_df[col].apply(lambda x: round(x, 2))
    cols = [col for col in agg_df.columns if 'arpd' in col or col == 'group']
    agg_df[cols].to_csv(filename, index=False)

In [30]:
export_arpd(dpm_merged, 'dpm-arpd-summary.csv')
export_arpd(dfm_merged, 'dfm-arpd-summary.csv')

In [21]:
agg_dataframe(dfm_merged)

,group,rel_gap_neh,arpd_neh,rel_gap_neh_ls,arpd_neh_ls,rel_gap_multistart,arpd_multistart,arpd_iga
0,1,4.857,4.857,2.255,2.255,0.351,0.351,0.151
1,2,5.307,5.307,3.755,3.755,0.444,0.444,0.168
2,3,5.248,4.429,3.383,2.571,1.174,0.384,0.194
3,4,3.163,3.073,2.068,1.980,0.242,0.154,0.117
4,5,5.382,4.509,3.452,2.598,1.847,1.004,0.470
5,6,11.645,4.466,10.129,3.044,7.235,0.338,-0.822


In [13]:
agg_dataframe(df_dfm[df_dfm["solved"]])

,group,lb1_ratio,lb5_ratio,gap,execution_time,nodes,solved
0,1,0.964068,0.969839,0.0,0.137402,2479.0,10
1,2,0.915220,0.923158,0.0,2.243004,66620.5,10
2,3,0.873408,0.897583,0.0,172.700473,3117077.0,9
3,4,0.985437,0.986126,0.0,0.395077,8266.0,9
4,5,0.975109,0.977976,0.0,185.433110,3344781.5,6


In [12]:
df_dpm.head()

,group,name,warmstart,lb1_start,lb5_start,incumbent,lower_bound,execution_time,gap,nodes,sequence,Group
0,1,dpm-001-20-5x6,1465,1326,1326,1381,1381,0.288417,0.0,12255,"[17, 10, 15, 2, 16, 19, 6, 18, 14, 12, 9, 5, 1...",1
1,1,dpm-002-20-5x6,1455,1328,1328,1385,1385,0.160825,0.0,9840,"[6, 3, 14, 8, 19, 0, 16, 1, 7, 11, 9, 5, 12, 4...",1
2,1,dpm-003-20-5x6,1559,1470,1470,1485,1485,0.011168,0.0,77,"[1, 16, 6, 7, 11, 17, 12, 3, 2, 14, 8, 4, 0, 1...",1
3,1,dpm-004-20-5x6,1519,1413,1413,1458,1458,0.121580,0.0,7669,"[9, 19, 12, 10, 2, 0, 15, 18, 4, 5, 7, 8, 3, 6...",1
4,1,dpm-005-20-5x6,1511,1371,1382,1465,1465,402.908985,0.0,21733552,"[0, 1, 2, 8, 12, 4, 16, 9, 10, 14, 15, 19, 13,...",1
